In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.FilterSamples import FilterSamples
from src.predictionModule.LoadupSamples import LoadupSamples

import pandas as pd
import numpy as np
import polars as pl
import datetime
import seaborn as sns
import lightgbm as lgb
import random
import matplotlib.pyplot as plt
import logging
import time
import re
import copy

formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
file_handler = logging.FileHandler(f"notebook-{formatted_date}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-07-13 00:12:58,983 - This will print to the notebook's output cell


In [2]:
params = {
    "idxAfterPrediction": 5,
    'timesteps': 25,
    'target_option': 'last',
    
    "TreeTime_lstm_units": 64,
    "TreeTime_lstm_num_layers": 3,
    "TreeTime_lstm_dropout": 0.00001,
    "TreeTime_lstm_recurrent_dropout": 0.00001,
    "TreeTime_lstm_learning_rate": 0.001,
    "TreeTime_lstm_optimizer": "adam",
    "TreeTime_lstm_bidirectional": True,
    "TreeTime_lstm_batch_size": 2**14,
    "TreeTime_lstm_epochs": 4,
    "TreeTime_lstm_l1": 0.00001,
    "TreeTime_lstm_l2": 0.00001,
    "TreeTime_inter_dropout": 0.00001,
    "TreeTime_input_gaussian_noise": 0.00001,
    "TreeTime_lstm_conv1d": True,
    "TreeTime_lstm_conv1d_kernel_size": 3,
    "TreeTime_lstm_loss": "mse",
}

In [3]:
stock_group = "group_snp500_finanTo2011"

eval_date = datetime.date(2025,  2,  1)

start_Date = datetime.date(year=2014, month=1, day=1)

ls = LoadupSamples(
    train_start_date=start_Date,
    test_dates=[eval_date],
    group=stock_group,
    params=params,
)

In [4]:
ls.load_samples(main_path="../src/featureAlchemy/bin/")

2025-07-13 00:14:22,859 - Test date 2025-02-01 not found in the database. Resetting to last trading day.


In [5]:
fs_params= {
    "FilterSamples_lincomb_q_up": 0.95,
    "FilterSamples_lincomb_lr": 0.0008384895113158295,
    "FilterSamples_lincomb_epochs": 800,
    "FilterSamples_lincomb_probs_noise_std": 0.010503627436184224,
    "FilterSamples_lincomb_subsample_ratio": 0.2424109747001177,
    "FilterSamples_lincomb_sharpness": 0.59226051089996,
    "FilterSamples_lincomb_featureratio": 0.33269072850403053,
    "FilterSamples_lincomb_itermax": 1,
    "FilterSamples_lincomb_show_progress": True,
}

In [6]:
dates = ls.meta_pl_train.select(pl.col("date")).to_series()
Xtree_all = ls.train_Xtree
ytree_all = ls.train_ytree

end_train_date = datetime.date(2024,  2,  1)
train_days = 390  # 2 years of training data
test_days = 30  # 1 month of testing data
        
def est_samples(X_all, y_all, dates: pl.Series, end_train_date, train_days, test_days):
    train_start = end_train_date - datetime.timedelta(days=train_days)
    train_end   = end_train_date

    test_start  = end_train_date + datetime.timedelta(days=1)
    test_end    = end_train_date + datetime.timedelta(days=test_days)

    # slice train
    train_mask = (
        (dates >= train_start) &
        (dates <= train_end)
    ).fill_null(False).to_numpy()

    # slice test
    test_mask = (
        (dates >= test_start) &
        (dates <= test_end)
    ).fill_null(False).to_numpy()

    return (
        X_all[train_mask],
        X_all[test_mask],
        y_all[train_mask],
        y_all[test_mask],
        dates.filter(train_mask),
        dates.filter(test_mask)
    )

In [7]:
end_train_dates = sorted([
    datetime.date(2024,  2,  1) - datetime.timedelta(days=i*60 + random.randint(-10,10)) 
    for i in range(50)
])

test_scores = []
for i, end_date in enumerate(end_train_dates):
    split_f = 0.90
    train_days = 390  # 2 years of training data
    test_days = int(train_days * (1-split_f)) 
    
    (
        Xtree_train,
        Xtree_test,
        ytree_train,
        ytree_test,
        samples_dates_train,
        samples_dates_test
    ) = est_samples(Xtree_all, ytree_all, dates, end_date, train_days=train_days, test_days=test_days)
            
    fs = FilterSamples(
        Xtree_train = Xtree_train,
        ytree_train = ytree_train,
        treenames = ls.featureTreeNames,
        Xtree_test = Xtree_test,
        samples_dates_train = samples_dates_train,
        samples_dates_test = samples_dates_test,
        ytree_test = ytree_test,
        params = fs_params
    )
    
    _, _, s_tr, s_te = fs.run()
    s_te = 1.0 if s_te is None or s_te<0.5 else s_te
    test_scores.append(s_te)
    
    logger.info(f"")
    logger.info(f"END DATE {end_date}")
    logger.info(f"Mean of all test scores: {np.mean([s for s in test_scores if ~np.isnan(s) or s is not None])}")
    logger.info("")
    
    logger.info(f"Training score: {(s_tr)}, testing score: {(s_te)}")



2025-07-13 00:14:50,401 - FilterSamples: (train) mean of y values 0.9989658184532801.
2025-07-13 00:14:50,401 - FilterSamples: (test) mean of y values 1.0079994197783235.
2025-07-13 00:14:50,401 - FilterSamples: Starting Lincomb
2025-07-13 00:14:50,418 -   FilterSamples: Lincomb Iteration 0/1 running.
2025-07-13 00:14:51,332 - FilterSamples: Best init score 1.0032


Epochs: 100%|██████████| 800/800 [00:04<00:00, 186.75it/s, mean_perdate_v=1.0071, mean_v=1.0054]


2025-07-13 00:14:57,004 -   Mean target (train): 1.010627632672761
2025-07-13 00:14:57,004 -   Number of features selected: 455
2025-07-13 00:14:57,004 -   Max distance between idces: 201
2025-07-13 00:14:57,014 -   Mean distance between idces: 19.99437851342911
2025-07-13 00:14:57,015 -   Median distance between idces: 14.0
2025-07-13 00:14:57,031 -   Mean target (test): 1.0112886424943435
2025-07-13 00:14:57,033 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-13 00:14:57,035 -   w quantile     : 9.35937213897705
2025-07-13 00:14:57,035 -   w_test quantile: 9.8912935256958
2025-07-13 00:14:57,036 -     Top-0 feature: FinData_quar_surprisePercentage
2025-07-13 00:14:57,037 -     Top-1 feature: FinData_quar_reportTime
2025-07-13 00:14:57,038 -     Top-2 feature: FeatureTA_trend_aroon_down
2025-07-13 00:14:57,038 -     Top-3 feature: Fourier_Price_SignCoeff_3_MH_1
2025-07-13 00:14:57,038 -     Top-4 feature: FinData_metrics_log_forward_pe_ratio
2025-07-13 00:14:57,038 -

Epochs: 100%|██████████| 800/800 [00:04<00:00, 193.70it/s, mean_perdate_v=1.0091, mean_v=1.0067]


2025-07-13 00:15:02,226 -   Mean target (train): 1.0126222963311298
2025-07-13 00:15:02,226 -   Number of features selected: 455
2025-07-13 00:15:02,226 -   Max distance between idces: 201
2025-07-13 00:15:02,226 -   Mean distance between idces: 19.999380037197767
2025-07-13 00:15:02,226 -   Median distance between idces: 14.0
2025-07-13 00:15:02,243 -   Mean target (test): 1.0076070004722621
2025-07-13 00:15:02,243 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-13 00:15:02,243 -   w quantile     : 10.120238304138184
2025-07-13 00:15:02,243 -   w_test quantile: 12.638519287109375
2025-07-13 00:15:02,243 -     Top-0 feature: FinData_quar_surprisePercentage
2025-07-13 00:15:02,243 -     Top-1 feature: FinData_quar_reportTime
2025-07-13 00:15:02,243 -     Top-2 feature: Fourier_Price_RSMERatioCoeff_1_MH_4
2025-07-13 00:15:02,243 -     Top-3 feature: Fourier_Price_SignCoeff_2_MH_12
2025-07-13 00:15:02,243 -     Top-4 feature: Fourier_Price_AmpCoeff_2_MH_12
2025-07-13 00:

...

2025-07-13 00:19:33,126 -   Mean target (train): 1.0190160680322817
2025-07-13 00:19:33,126 -   Number of features selected: 455
2025-07-13 00:19:33,126 -   Max distance between idces: 230
2025-07-13 00:19:33,126 -   Mean distance between idces: 19.971481711097333
2025-07-13 00:19:33,130 -   Median distance between idces: 9.0
2025-07-13 00:19:33,147 -   Mean target (test): 0.9994309494869601
2025-07-13 00:19:33,151 -   Fraction of days with no coverage by test mask: 3.70%
2025-07-13 00:19:33,153 -   w quantile     : 15.7525634765625
2025-07-13 00:19:33,154 -   w_test quantile: 10.205588340759277
2025-07-13 00:19:33,154 -     Top-0 feature: Fourier_Price_RSMECoeff_4_MH_8
2025-07-13 00:19:33,154 -     Top-1 feature: FinData_ann_ebit_nivRevLag_am3
2025-07-13 00:19:33,154 -     Top-2 feature: Fourier_Price_SignCoeff_1_MH_12
2025-07-13 00:19:33,157 -     Top-3 feature: FinData_quar_commonStockSharesOutstanding_nivRev
2025-07-13 00:19:33,157 -     Top-4 feature: Fourier_Price_RSMERatioCoeff_

Epochs: 100%|██████████| 800/800 [00:04<00:00, 171.10it/s, mean_perdate_v=1.0126, mean_v=1.0118]


2025-07-13 00:19:38,907 -   Mean target (train): 1.0177521545712498
2025-07-13 00:19:38,907 -   Number of features selected: 455
2025-07-13 00:19:38,907 -   Max distance between idces: 239
2025-07-13 00:19:38,907 -   Mean distance between idces: 19.979464841319228
2025-07-13 00:19:38,907 -   Median distance between idces: 9.0
2025-07-13 00:19:38,925 -   Mean target (test): 0.9952326611504645
2025-07-13 00:19:38,927 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-13 00:19:38,929 -   w quantile     : 14.840120315551758
2025-07-13 00:19:38,932 -   w_test quantile: 11.858430862426758
2025-07-13 00:19:38,933 -     Top-0 feature: Fourier_ReturnLog_RSMECoeff_3_MH_6
2025-07-13 00:19:38,934 -     Top-1 feature: FinData_quar_operatingCashflow_nivRevLag_qm5
2025-07-13 00:19:38,935 -     Top-2 feature: FinData_quar_ebit_nivRevLag_qm6
2025-07-13 00:19:38,935 -     Top-3 feature: Fourier_Price_RSMERatioCoeff_1_MH_6
2025-07-13 00:19:38,936 -     Top-4 feature: FinData_quar_grossProf

Epochs: 100%|██████████| 800/800 [00:04<00:00, 166.32it/s, mean_perdate_v=1.0129, mean_v=1.0138]


2025-07-13 00:19:44,834 -   Mean target (train): 1.0179279383948534
2025-07-13 00:19:44,834 -   Number of features selected: 455
2025-07-13 00:19:44,834 -   Max distance between idces: 239
2025-07-13 00:19:44,834 -   Mean distance between idces: 20.003123048094942
2025-07-13 00:19:44,834 -   Median distance between idces: 10.0
2025-07-13 00:19:44,853 -   Mean target (test): 1.0098458066137386
2025-07-13 00:19:44,855 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-13 00:19:44,856 -   w quantile     : 14.188985824584961
2025-07-13 00:19:44,858 -   w_test quantile: 11.310796737670898
2025-07-13 00:19:44,859 -     Top-0 feature: MathFeature_Drawup_MH12
2025-07-13 00:19:44,860 -     Top-1 feature: FinData_quar_ebit_nivRevLag_qm6
2025-07-13 00:19:44,861 -     Top-2 feature: FinData_quar_grossProfit_nivRevLag_qm6
2025-07-13 00:19:44,861 -     Top-3 feature: FinData_quar_grossProfit_nivRevLag_qm7
2025-07-13 00:19:44,862 -     Top-4 feature: FinData_quar_reportTime
2025-07-13 